<a href="https://colab.research.google.com/github/sid-rawal/Clinical-Sentence-Segmentation/blob/main/BERT_Sentence_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simpletransformers Segmenter

In [ ]:
!pip install simpletransformers

##Train BERT model

Code for training on MIMIC or Brown dataset.

Annotated/preprocessed files are loaded from Drive.

In [ ]:
# Replace "B" with "B-SENT"
new_f = open('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/brown_test_windows_fulltags.txt', 'w')
with open('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/brown_test_windows.txt', 'r') as orig_f:
  for line in orig_f:
    if line != '\n':
      tok, tag = line.split()
      if tag.rstrip() == 'B':
        new_f.write(' '.join([tok, 'B-SENT\n']))
      elif tag.rstrip() == 'I':
        new_f.write(' '.join([tok, 'I-SENT\n']))
      else:
        new_f.write(line)
    else:
      new_f.write(line)

new_f.close()

In [ ]:
from simpletransformers.ner import NERModel

# custom_labels = ["O", "B", "I"]
custom_labels = ["O", "B-SENT", "I-SENT"]

model = NERModel(
    "bert", "bert-base-cased", labels=custom_labels, args={'learning_rate': 2e-5,
                                                           'num_train_epochs': 2,
                                                           'max_seq_length': 512,
                                                           'overwrite_output_dir': True,
                                                           'reprocess_input_data': True,
                                                           "output_dir": "/content/drive/MyDrive/Penn/mimic iii/simpletransformers/brown_bert_base_cased/",
                                                           "cache_dir": "/content/drive/MyDrive/Penn/mimic iii/simpletransformers/cache_dir/"
                                                           }
)

model.train_model('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/brown_train_windows_fulltags.txt')
print('Finished training')

eval_result, model_outputs, wrong_preds = model.eval_model('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/brown_val_windows_fulltags.txt')
print('Eval results:', eval_result)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

...

Running Epoch 1 of 2:   0%|          | 0/4507 [00:00<?, ?it/s]

Finished training


  0%|          | 0/2 [00:01<?, ?it/s]

Running Evaluation:   0%|          | 0/1137 [00:00<?, ?it/s]

Eval results: {'eval_loss': 0.012605768749674708, 'precision': 0.952571123347028, 'recall': 0.9361539291776642, 'f1_score': 0.9442911754153728}


##Load and Evaluate the model

In [ ]:
from simpletransformers.ner import NERModel

custom_labels = ["O", "B-SENT", "I-SENT"]
model = NERModel('bert', '/content/drive/MyDrive/Penn/mimic iii/simpletransformers/brown_bert_base_cased/', labels=custom_labels)
eval_result, model_outputs, wrong_preds = model.eval_model('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/brown_val.txt')

print(eval_result)

## MIMIC evaluation results:
# Results for model trained after correcting deidentified regex to non-greedy (128 windowsize model)
# {'eval_loss': 0.12261740378408621, 'precision': 0.8442914021306359, 'recall': 0.8597413982029367, 'f1_score': 0.8519463597372279}

# Results for model trained on maximum window size (512)
# {'eval_loss': 0.15730027976550756, 'precision': 0.6950020766994324, 'recall': 0.7707069931680356, 'f1_score': 0.7308994285298293}

In [ ]:
predictions, raw_output = model.predict(['This is a <newline> test <newline> name of practitioner: <deidentified> ____ <newline> This is another test.'])
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'This': 'B-SENT'},
  {'is': 'I-SENT'},
  {'a': 'I-SENT'},
  {'<newline>': 'I-SENT'},
  {'test': 'I-SENT'},
  {'<newline>': 'I-SENT'},
  {'name': 'I-SENT'},
  {'of': 'I-SENT'},
  {'practitioner:': 'I-SENT'},
  {'<deidentified>': 'I-SENT'},
  {'____': 'I-SENT'},
  {'<newline>': 'I-SENT'},
  {'This': 'B-SENT'},
  {'is': 'I-SENT'},
  {'another': 'I-SENT'},
  {'test.': 'I-SENT'}]]

## Load test data

In [ ]:
# Go through each note file and save each file name with its text.
import pandas as pd
import os

test_notes = []
# for root, dirs, files in os.walk('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/mimic/test'):
for root, dirs, files in os.walk('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/val'):
    for name in files:
        with open(os.path.join(root, name)) as f:
            note_text = f.read()
            test_notes.append({'name': name, 'text': note_text})
  
test_df = pd.DataFrame(test_notes)    
test_df

,name,text
0,cb08.txt,"Old, tired, trembling the woman came to the ca..."
1,cc16.txt,A year ago it was bruited that the primary cha...
2,cc12.txt,Radio is easily outdistancing television in it...
3,cb07.txt,The U.N.'s' gravest crisis' Ambassador Stevens...
4,cd07.txt,"""The Lord is my light and my salvation;; whom ..."
...,...,...
75,ca36.txt,Sen. John L. McClellan of Arkansas and Rep. Da...
76,ca37.txt,The nation the three-front war At a closed-doo...
77,ca41.txt,A philosopher may point out that the troubles ...
78,cb02.txt,Must Berlin remain divided?? The inference has...


##Predict on test data

In [ ]:
def predict_windows(notes_df, model):
    # Input is a DataFrame with note name and note text
    # Output: a list of lists of tuples (token, tag)
    # Sliding window predictions
    final_preds = []
    notes = notes_df['text'].to_list()
    note_names = notes_df['name'].to_list()
    for idx, note in enumerate(notes):#, pred_note2]:
        end_of_note = False
        left_ptr = 0
        # Don't need right pointer -- model will only return maximum seq length
        note_words = note.split()
        # Initialize predictions to 'O'
        note_preds = [[tok, 'O'] for tok in note_words]
        while not end_of_note:
            to_predict = ' '.join(note_words[left_ptr:])
            window_preds = model.predict([to_predict])[0][0]
            # Manually set the first prediction to be 'B' -- this is in case there was no sentence break predicted in the previous sequence
            list(window_preds[0].values())[0] = 'B-SENT'

            # Update the note predictions with the predictions for this window
            for i in range(len(window_preds)):
              note_preds[left_ptr+i][1] = list(window_preds[i].values())[0]

            if len(window_preds) == 1:
              end_of_note = True
            # Update left pointer
            for i in range(1, len(window_preds)):
                if left_ptr + i >= len(note_words):
                  end_of_note = True
                  break
                else:
                  if list(window_preds[i].values())[0] == 'B-SENT':
                    # print('breaking on:', window_preds[i])
                    break

            if i == len(window_preds):
              print('Overlong sequence',  list(window_preds[0].values())[0])

            left_ptr += i

        note_preds = [(tok, tag) for tok, tag in note_preds]
        final_preds.append({'title': note_names[idx], 'predictions': note_preds})
        print(f'Finished {idx+1}/{len(notes_df)} notes...')

    return final_preds


In [ ]:
test_df

,name,text
0,cb08.txt,"Old, tired, trembling the woman came to the ca..."
1,cc16.txt,A year ago it was bruited that the primary cha...
2,cc12.txt,Radio is easily outdistancing television in it...
3,cb07.txt,The U.N.'s' gravest crisis' Ambassador Stevens...
4,cd07.txt,"""The Lord is my light and my salvation;; whom ..."
...,...,...
75,ca36.txt,Sen. John L. McClellan of Arkansas and Rep. Da...
76,ca37.txt,The nation the three-front war At a closed-doo...
77,ca41.txt,A philosopher may point out that the troubles ...
78,cb02.txt,Must Berlin remain divided?? The inference has...


In [ ]:
%%capture --no-stdout
# ^ We want to hide the progress bars since there will be a lot (we are predicting on one window at a time) 

test_preds = predict_windows(test_df, model)

Finished 1/80 notes...
Finished 2/80 notes...
Finished 3/80 notes...
Finished 4/80 notes...
Finished 5/80 notes...
Finished 6/80 notes...
Finished 7/80 notes...
Finished 8/80 notes...
Finished 9/80 notes...
Finished 10/80 notes...
Finished 11/80 notes...
Finished 12/80 notes...
Finished 13/80 notes...
Finished 14/80 notes...
Finished 15/80 notes...
Finished 16/80 notes...
Finished 17/80 notes...
Finished 18/80 notes...
Finished 19/80 notes...
Finished 20/80 notes...
Finished 21/80 notes...
Finished 22/80 notes...
Finished 23/80 notes...
Finished 24/80 notes...
Finished 25/80 notes...
Finished 26/80 notes...
Finished 27/80 notes...
Finished 28/80 notes...
Finished 29/80 notes...
Finished 30/80 notes...
Finished 31/80 notes...
Finished 32/80 notes...
Finished 33/80 notes...
Finished 34/80 notes...
Finished 35/80 notes...
Finished 36/80 notes...
Finished 37/80 notes...
Finished 38/80 notes...
Finished 39/80 notes...
Finished 40/80 notes...
Finished 41/80 notes...
Finished 42/80 notes...
F

In [ ]:
test_preds_df = pd.DataFrame(test_preds)
test_preds_df

In [ ]:
test_preds_df['title'].tolist()

In [ ]:
from tqdm import tqdm
import re

all_spans = []
for i, row in test_preds_df.iterrows():
  # Put code for character-level predictions here
  chars = []
  tags = []
  with open(f'/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/val_notes_character_BIO/{row["title"]}', 'r') as f:
      for line in f:
          if len(line.split()) == 3:
            _, ch, tag = line.split()
          else:
            _, tag = line.split()
            ch = ' '
          chars.append(ch)
          tags.append(tag)

  note_raw = ''.join(chars)
  sent_spans = []
  pos = 0
  isSentence = False
  note_preds = row['predictions']
  # for tok, tag in note_preds:
  #     if tok == '<newline>':
  #       note_preds_final.append(('\\\\n', tag))
  #     else:
  #       note_preds_final.append((tok, tag))
  for tok, tag in tqdm(note_preds):
      tok_final = tok
      if tok == '<deidentified>':
          for match in re.finditer('\[\*\*(.*?)\*\*\]', note_raw):
            # print(match.start(), match.end())
            if match.start() >= pos:
              break
      elif tok == '<newline>':
          for match in re.finditer('\\n', note_raw):
            if match.start() >= pos:
              break
      else:
          for match in re.finditer(re.escape(tok_final), note_raw):
            if match.start() >= pos:
              break
          if not re.search(re.escape(tok_final), note_raw):
            print('Error: could not find token:', re.escape(tok_final))
            print(note_raw)

      begin, end = match.start(), match.end()
      
      if tag == 'B-SENT':
        if isSentence:
          sent_spans.append((prev_begin, prev_end))
        isSentence = True
        prev_begin, prev_end = match.start(), match.end()
      elif tag == 'O':
        if isSentence:
          sent_spans.append((prev_begin, prev_end))
        isSentence = False
      elif tag == 'I-SENT':
          if not isSentence:
            prev_begin = match.start()
          prev_end = match.end()
          isSentence = True

      pos = match.end()

  all_spans.append(sent_spans)
  # print(sent_spans)
  curr_span = 0
  tok_tags = []
  for i, ch in enumerate(note_raw):
      # Correct the newlines to show up as one character instead of \ and n separately
      if tok_tags and tok_tags[-1][0] == '\\' and ch == 'n':
        tok_tags[-1] = ('\\n', tok_tags[-1][1])
        continue
      if i == sent_spans[curr_span][0]:
        # print(ch, 'B')
        tok_tags.append((ch, 'B'))
      elif i > sent_spans[curr_span][0] and i < sent_spans[curr_span][1]:
        # print(ch, 'I')
        tok_tags.append((ch, 'I'))
      else:
        if curr_span < len(sent_spans)-1 and i == sent_spans[curr_span+1][0]:
          curr_span += 1
          # print(ch, 'B')
          tok_tags.append((ch, 'B'))
        else:
          # print(ch, 'O')
          tok_tags.append((ch, 'O'))
      
  with open(f'/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/val_notes_predictions/{row["title"]}', 'w') as f:
    f.write('\n'.join([tok + ' ' + tag for tok, tag in tok_tags]))
  
      


In [ ]:
import pandas as pd
# test_preds_df.to_csv('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/mimic/test_predictions.tsv', sep='\t', index=False)
test_preds_df.to_csv('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/val_predictions.tsv', sep='\t', index=False)

In [ ]:
test_preds_df.title.tolist()

## Evaluate

Token-level prediction (because BERT only uses tokens, so it should be the same results as character-level otherwise).

First, write the gold label tokens/tags and prediction tokens/tags to a single .txt file in the same file order.

In [ ]:
import pandas as pd
from ast import literal_eval

# test_preds_df = pd.read_csv('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/test_predictions_512_window.tsv', sep='\t')
test_preds_df = pd.read_csv('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/val_predictions.tsv', sep='\t')


# # Writing gold labels file
# # Note: gold labels file will have the first column as the index of the sentence beginning. This can be ignored -- it's just for debugging
# with open('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/test_tokenized_gold_labels.txt', 'w') as final_f:
#     for filename in test_preds_df['title']:
#         with open(f'/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/test_notes_tokenized_tagged/{filename}', 'r') as note_f:
#             gold_note = ''
#             for line in note_f.readlines():
#                 _, tok, tag = line.split()
#                 gold_note += ' '.join([tok, tag]) + '\n'
#                 # gold_note.write(tok + ' ' + tag + '\n')
#                 # gold_note = note_f.read()
#         final_f.write(gold_note)
#         final_f.write('\n')

# Writing predictions file
with open('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/val_tokenized_predictions.txt', 'w') as f:
# with open('/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/test_tokenized_predictions.txt', 'w') as f:
    for note_preds in test_preds_df['predictions']:
        # Last token/tag is an extra newline, it is not present in the gold labels so exclude it
        # Note: the above is not true for the Brown corpus!!
        for tok, tag in literal_eval(note_preds):#[:-1]:
            if tag == 'B-SENT':
                f.write(tok + ' B\n')
            elif tag == 'I-SENT':
                f.write(tok + ' I\n')
            else:
                f.write(tok + ' ' + tag + '\n')
        f.write('\n')



In [ ]:
def evaluate_strict(gold_file, pred_file):
    gold_labels, pred_labels = [], []
    with open(gold_file, 'r') as gold_f:
        note_num = 0
        line_num = 1
        begin_line, end_line = 0, 0
        for line in gold_f:
            if line == '\n':
                note_num += 1
                continue
            tag = line.strip()[-1]
            if tag == 'B':
                if begin_line != -1:
                    gold_labels.append((begin_line, end_line))
                begin_line = line_num
            elif tag == 'I':
                end_line = line_num
            elif tag == 'O':
                if begin_line != -1:
                    gold_labels.append((begin_line, end_line))
                begin_line = -1
            line_num += 1

    with open(pred_file, 'r') as pred_f:
        note_num = 0
        line_num = 1
        begin_line, end_line = 0, 0
        for line in pred_f:
            if line == '\n':
                note_num += 1
                continue
            tag = line.strip()[-1]
            if tag == 'B':
                if begin_line != -1:
                    pred_labels.append((begin_line, end_line))
                begin_line = line_num
            elif tag == 'I':
                end_line = line_num
            elif tag == 'O':
                if begin_line != -1:
                    pred_labels.append((begin_line, end_line))
                begin_line = -1
            line_num += 1

    # print(gold_labels)
    # print(pred_labels)

    gold_labels = set(gold_labels)
    pred_labels = set(pred_labels)
    FP = len(pred_labels - gold_labels)
    # Subtracting 1 since the first span is (0, 0)
    TP = len(pred_labels & gold_labels) - 1
    FN = len(gold_labels - pred_labels)

    print('FP:', FP)
    print('TP:', TP)
    print('FN:', FN)

In [ ]:
evaluate_strict(gold_file=r'/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/test_tokenized_gold_labels.txt',
                pred_file=r'/content/drive/MyDrive/Penn/mimic iii/simpletransformers/corpus/brown/test_tokenized_predictions.txt')

FP: 428
TP: 11392
FN: 404
